## rotatiehoek

In [1]:
import math
import xml.etree.ElementTree as ET
# === 1. Configuratie ===
CAMERAS_XML = r"C:\Users\Sander\OneDrive - UGent\Semester_2\Masterproef\Thesis_ML\Roboflow\Raw_July\All_results\ViT\mapping_back\cameras.xml"

# De exacte camera‐labels zoals in je XML:
FIRST_LABEL = "DJI_20240730173659_0091_D"
LAST_LABEL  = "DJI_20240730173812_0173_D"

# === 2. XML inlezen ===
tree = ET.parse(CAMERAS_XML)
root = tree.getroot()

def get_lon_lat(label):
    """Haal de reference x,y (lon,lat) uit de <camera label="…"> entry."""
    cam = root.find(f".//cameras/camera[@label='{label}']")
    if cam is None:
        raise KeyError(f"Camera with label {label} not found in XML.")
    ref = cam.find("reference")
    if ref is None:
        raise KeyError(f"<reference> tag missing for camera {label}.")
    lon = float(ref.get("x"))
    lat = float(ref.get("y"))
    return lon, lat

# === 3. Coördinaten ophalen ===
lon1, lat1 = get_lon_lat(FIRST_LABEL)
lon2, lat2 = get_lon_lat(LAST_LABEL)

# === 4. Hoek berekenen t.o.v. x-as (oost-richting) ===
delta_lon = lon2 - lon1
delta_lat = lat2 - lat1

# atan2(dY, dX) → hoek in radialen, 0° = oost, +90° = noorden
angle_rad = math.atan2(delta_lat, delta_lon)
angle_deg = math.degrees(angle_rad)

print(f"Rotatiehoek t.o.v. x-as: {angle_deg:.2f}°")

Rotatiehoek t.o.v. x-as: -23.67°


## coordinaten box

In [2]:
import math
import pandas as pd
import os

# Now merge
merged_df = pd.read_csv(r"C:\Users\Sander\OneDrive - UGent\Semester_2\Masterproef\Thesis_ML\Roboflow\Raw_July\All_results\ViT\metadata_full_July_updated.csv")

# Camera parameters
fly_height = 25.026  # in meters²&az
focal_length = 12.29 / 1000  # in meters (convert mm to meters)
sensor_width = 17.73 / 1000  # in meters (convert mm to meters)

# Calculate the ground sampling distance (GSD) in meters per pixel
# GSD = (sensor_width * fly_height) / (focal_length * image_width)
# Assuming all images have the same width and height
image_width = 5280
image_height = 3956  # or the correct height of your stitched image

# Corrected GSD (still assumes square pixels)
gsd = (sensor_width * fly_height) / (focal_length * image_width)

# Updated helper function
def pixel_to_gps(row, x_pixel, y_pixel):
    lat_center = row["latitude"]
    lon_center = row["longitude"]
    heading_deg = row.get("Yaw", 270 + angle_deg)  # default to 0 if missing

    # Offset from image center
    x_offset_m = (x_pixel - image_width / 2) * gsd
    y_offset_m = (y_pixel - image_height / 2) * gsd

    # Rotate offset by heading
    theta = math.radians(-heading_deg)  # negative for clockwise rotation
    x_rot = x_offset_m * math.cos(theta) - y_offset_m * math.sin(theta)
    y_rot = x_offset_m * math.sin(theta) + y_offset_m * math.cos(theta)

    # Convert to degrees
    lat_offset_deg = -y_rot / 111320  # y_rot affects latitude
    lon_offset_deg = x_rot / (111320 * math.cos(math.radians(lat_center)))

    lat_new = lat_center + lat_offset_deg
    lon_new = lon_center + lon_offset_deg

    return lat_new, lon_new


# Create new columns for GPS coordinates of bounding box corners
for index, row in merged_df.iterrows():
    x_min, y_min, x_max, y_max = row["x_start"], row["y_start"], row["x_start"] + 128, row["y_start"] + 128

    # Convert each corner to GPS coordinates
    lat_min, lon_min = pixel_to_gps(row, x_min, y_min)
    lat_max, lon_max = pixel_to_gps(row, x_max, y_max)

    # Add the new GPS coordinates to the DataFrame
    merged_df.at[index, "Lat_min"] = lat_min
    merged_df.at[index, "Lon_min"] = lon_min
    merged_df.at[index, "Lat_max"] = lat_max
    merged_df.at[index, "Lon_max"] = lon_max

# Save the updated DataFrame to a new CSV file
gps_bounding_boxes_file = r"C:\Users\Sander\OneDrive - UGent\Semester_2\Masterproef\Thesis_ML\Roboflow\Raw_July\All_results\ViT\metadata_full_July_updated_coord.csv"
merged_df.to_csv(gps_bounding_boxes_file, index=False)

print(f"Bounding boxes with GPS coordinates saved to '{gps_bounding_boxes_file}'")

Bounding boxes with GPS coordinates saved to 'C:\Users\Sander\OneDrive - UGent\Semester_2\Masterproef\Thesis_ML\Roboflow\Raw_July\All_results\ViT\metadata_full_July_updated_coord.csv'


## Geojson aanmaken

In [3]:
import geopandas as gpd
from shapely.affinity import rotate as affinity_rotate
from shapely.geometry import box

affinity = type("affinity", (), {"rotate": affinity_rotate})
# ─── 1. LOAD CSV ──────────────────────────────────────────────────────────────
CSV_IN  = r"C:\Users\Sander\OneDrive - UGent\Semester_2\Masterproef\Thesis_ML\Roboflow\Raw_July\All_results\ViT\metadata_full_July_updated_coord.csv"
GEO_OUT = r"C:\Users\Sander\OneDrive - UGent\Semester_2\Masterproef\Thesis_ML\Roboflow\Raw_July\All_results\ViT\metadata_full_July_updated_coord.geojson"


df = pd.read_csv(CSV_IN)

# compute center longitude/latitude for each patch
df['Lon_center'] = (df['Lon_min'] + df['Lon_max']) / 2
df['Lat_center'] = (df['Lat_min'] + df['Lat_max']) / 2

# filter for your segment if desired:
df = df[df["segment"]=="F15"].reset_index(drop=True)

# you must have computed Lon_center, Lat_center in your CSV already
# patch_size in pixels (e.g. 128)
PATCH_PX = 128

# ─── 2. COMPUTE A SINGLE GSD (m/px) ──────────────────────────────────────────
# pick whichever axis or average makes sense; here we take the mean of your
# previously computed GSD_x / GSD_y if you have them, otherwise recompute:
# e.g.
# gsd = (df["GSD_x"].mean() + df["GSD_y"].mean())/2
# or if you only computed one:
# gsd = df["GSD_x"].mean()

# for example, hard-code from your camera:
# SENSOR_WIDTH_MM = 13.2
# IMAGE_WIDTH_PX  = 5280
# FLIGHT_ALT_M    = df["flight_altitude"].mean()
# gsd = (SENSOR_WIDTH_MM/1000 * FLIGHT_ALT_M) / IMAGE_WIDTH_PX

# *** replace the line below with your real number: ***
fly_height = 25.026  # in meters²&az
focal_length = 12.29 / 1000  # in meters (convert mm to meters)
sensor_width = 17.73 / 1000  # in meters (convert mm to meters)

# Calculate the ground sampling distance (GSD) in meters per pixel
# GSD = (sensor_width * fly_height) / (focal_length * image_width)
# Assuming all images have the same width and height
image_width = 5280
image_height = 3956  # or the correct height of your stitched image

# Corrected GSD (still assumes square pixels)
gsd = (sensor_width * fly_height) / (focal_length * image_width)

half_m = PATCH_PX * gsd / 2

# ─── 3. PROJECT CENTERS TO METRIC CRS ────────────────────────────────────────
# choose a local CRS where distances are in metres (e.g. your UTM zone)
METRIC_CRS = "EPSG:31370"
centers = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df.Lon_center, df.Lat_center),
    crs="EPSG:4326"
).to_crs(METRIC_CRS)

# ─── 4. BUILD PERFECT SQUARES AROUND EACH CENTER ─────────────────────────────
centers["geometry"] = centers.geometry.apply(
    lambda pt: box(pt.x-half_m, pt.y-half_m, pt.x+half_m, pt.y+half_m)
)

# ─── 5. ROTATE THE WHOLE BLOCK AS ONE ────────────────────────────────────────
# find the centroid of the union
minx, miny, maxx, maxy = centers.total_bounds
cx, cy = (minx+maxx)/2, (miny+maxy)/2

# your previously computed rotation angle (° CCW from +X)
angle_deg = -30 # ← fill in from your atan2 step

centers["geometry"] = centers.geometry.apply(
    lambda geom: affinity.rotate(geom, angle_deg, origin=(cx, cy))
)

# ─── 6. EXPORT ──────────────────────────────────────────────────────────────
# if you want a projected GeoJSON:
# centers.to_file(GEO_OUT, driver="GeoJSON")

# or to reproject back to WGS84:
centers.to_crs("EPSG:4326") \
        .to_file(GEO_OUT, driver="GeoJSON")

print("✅ Saved perfectly‐tessellated, rotated patches to:", GEO_OUT)

✅ Saved perfectly‐tessellated, rotated patches to: C:\Users\Sander\OneDrive - UGent\Semester_2\Masterproef\Thesis_ML\Roboflow\Raw_July\All_results\ViT\metadata_full_July_updated_coord.geojson


## option 2

In [4]:
# === 1. Configuratie ===
CAMERAS_XML = r"C:\Users\Sander\OneDrive - UGent\Semester_2\Masterproef\Thesis_ML\Roboflow\Raw_July\All_results\ViT\mapping_back\cameras.xml"

# De exacte camera‐labels zoals in je XML:
FIRST_LABEL = "DJI_20240730173659_0091_D"
LAST_LABEL  = "DJI_20240730173812_0173_D"

# === 2. XML inlezen ===
tree = ET.parse(CAMERAS_XML)
root = tree.getroot()

def get_lon_lat(label):
    """Haal de reference x,y (lon,lat) uit de <camera label="…"> entry."""
    cam = root.find(f".//cameras/camera[@label='{label}']")
    if cam is None:
        raise KeyError(f"Camera with label {label} not found in XML.")
    ref = cam.find("reference")
    if ref is None:
        raise KeyError(f"<reference> tag missing for camera {label}.")
    lon = float(ref.get("x"))
    lat = float(ref.get("y"))
    return lon, lat

# === 3. Coördinaten ophalen ===
lon1, lat1 = get_lon_lat(FIRST_LABEL)
lon2, lat2 = get_lon_lat(LAST_LABEL)

# === 4. Hoek berekenen t.o.v. x-as (oost-richting) ===
delta_lon = lon2 - lon1
delta_lat = lat2 - lat1

# atan2(dY, dX) → hoek in radialen, 0° = oost, +90° = noorden
angle_rad = math.atan2(delta_lat, delta_lon)
angle_deg = math.degrees(angle_rad)

print(f"Rotatiehoek t.o.v. x-as: {angle_deg:.2f}°")
import math
import pandas as pd
import os

# Now merge
merged_df = pd.read_csv(r"C:\Users\Sander\OneDrive - UGent\Semester_2\Masterproef\Thesis_ML\Roboflow\Raw_July\All_results\ViT\metadata_full_July_updated.csv")

# Camera parameters
fly_height = 25.026  # in meters²&az
focal_length = 12.29 / 1000  # in meters (convert mm to meters)
sensor_width = 17.73 / 1000  # in meters (convert mm to meters)

# Calculate the ground sampling distance (GSD) in meters per pixel
# GSD = (sensor_width * fly_height) / (focal_length * image_width)
# Assuming all images have the same width and height
image_width = 5280
image_height = 3956  # or the correct height of your stitched image

# Corrected GSD (still assumes square pixels)
gsd = (sensor_width * fly_height) / (focal_length * image_width)

# Updated helper function
def pixel_to_gps(row, x_pixel, y_pixel):
    lat_center = row["latitude"]
    lon_center = row["longitude"]
    heading_deg = row.get("Yaw", 270 + angle_deg)  # default to 0 if missing

    # Offset from image center
    x_offset_m = (x_pixel - image_width / 2) * gsd
    y_offset_m = (y_pixel - image_height / 2) * gsd

    # Rotate offset by heading
    theta = math.radians(-heading_deg)  # negative for clockwise rotation
    x_rot = x_offset_m * math.cos(theta) - y_offset_m * math.sin(theta)
    y_rot = x_offset_m * math.sin(theta) + y_offset_m * math.cos(theta)

    # Convert to degrees
    lat_offset_deg = -y_rot / 111320  # y_rot affects latitude
    lon_offset_deg = x_rot / (111320 * math.cos(math.radians(lat_center)))

    lat_new = lat_center + lat_offset_deg
    lon_new = lon_center + lon_offset_deg

    return lat_new, lon_new


# Create new columns for GPS coordinates of bounding box corners
for index, row in merged_df.iterrows():
    x_min, y_min, x_max, y_max = row["x_start"], row["y_start"], row["x_start"] + 128, row["y_start"] + 128

    # Convert each corner to GPS coordinates
    lat_min, lon_min = pixel_to_gps(row, x_min, y_min)
    lat_max, lon_max = pixel_to_gps(row, x_max, y_max)

    # Add the new GPS coordinates to the DataFrame
    merged_df.at[index, "Lat_min"] = lat_min
    merged_df.at[index, "Lon_min"] = lon_min
    merged_df.at[index, "Lat_max"] = lat_max
    merged_df.at[index, "Lon_max"] = lon_max

# Save the updated DataFrame to a new CSV file
gps_bounding_boxes_file = r"C:\Users\Sander\OneDrive - UGent\Semester_2\Masterproef\Thesis_ML\Roboflow\Raw_July\All_results\ViT\metadata_full_July_updated_coord.csv"
merged_df.to_csv(gps_bounding_boxes_file, index=False)

print(f"Bounding boxes with GPS coordinates saved to '{gps_bounding_boxes_file}'")
# ─── 1. LOAD CSV ──────────────────────────────────────────────────────────────
CSV_IN  = r"C:\Users\Sander\OneDrive - UGent\Semester_2\Masterproef\Thesis_ML\Roboflow\Raw_July\All_results\ViT\metadata_full_July_updated_coord.csv"
GEO_OUT = r"C:\Users\Sander\OneDrive - UGent\Semester_2\Masterproef\Thesis_ML\Roboflow\Raw_July\All_results\ViT\metadata_full_July_updated_coord.geojson"


df = pd.read_csv(CSV_IN)

# compute center longitude/latitude for each patch
df['Lon_center'] = (df['Lon_min'] + df['Lon_max']) / 2
df['Lat_center'] = (df['Lat_min'] + df['Lat_max']) / 2

# filter for your segment if desired:
df = df[df["segment"]=="F15"].reset_index(drop=True)

# you must have computed Lon_center, Lat_center in your CSV already
# patch_size in pixels (e.g. 128)
PATCH_PX = 128

# ─── 2. COMPUTE A SINGLE GSD (m/px) ──────────────────────────────────────────
# pick whichever axis or average makes sense; here we take the mean of your
# previously computed GSD_x / GSD_y if you have them, otherwise recompute:
# e.g.
# gsd = (df["GSD_x"].mean() + df["GSD_y"].mean())/2
# or if you only computed one:
# gsd = df["GSD_x"].mean()

# for example, hard-code from your camera:
# SENSOR_WIDTH_MM = 13.2
# IMAGE_WIDTH_PX  = 5280
# FLIGHT_ALT_M    = df["flight_altitude"].mean()
# gsd = (SENSOR_WIDTH_MM/1000 * FLIGHT_ALT_M) / IMAGE_WIDTH_PX

# *** replace the line below with your real number: ***
fly_height = 25.026  # in meters²&az
focal_length = 12.29 / 1000  # in meters (convert mm to meters)
sensor_width = 17.73 / 1000  # in meters (convert mm to meters)

# Calculate the ground sampling distance (GSD) in meters per pixel
# GSD = (sensor_width * fly_height) / (focal_length * image_width)
# Assuming all images have the same width and height
image_width = 5280
image_height = 3956  # or the correct height of your stitched image

# Corrected GSD (still assumes square pixels)
gsd = (sensor_width * fly_height) / (focal_length * image_width)

half_m = PATCH_PX * gsd / 2

# ─── 3. PROJECT CENTERS TO METRIC CRS ────────────────────────────────────────
# choose a local CRS where distances are in metres (e.g. your UTM zone)
METRIC_CRS = "EPSG:31370"
centers = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df.Lon_center, df.Lat_center),
    crs="EPSG:4326"
).to_crs(METRIC_CRS)

# ─── 4. BUILD PERFECT SQUARES AROUND EACH CENTER ─────────────────────────────
centers["geometry"] = centers.geometry.apply(
    lambda pt: box(pt.x-half_m, pt.y-half_m, pt.x+half_m, pt.y+half_m)
)

# ─── 5. ROTATE THE WHOLE BLOCK AS ONE ────────────────────────────────────────
# find the centroid of the union
minx, miny, maxx, maxy = centers.total_bounds
cx, cy = (minx+maxx)/2, (miny+maxy)/2

# your previously computed rotation angle (° CCW from +X)
angle_deg = -2 # ← fill in from your atan2 step

centers["geometry"] = centers.geometry.apply(
    lambda geom: affinity.rotate(geom, angle_deg, origin=(cx, cy))
)

# ─── 6. EXPORT ──────────────────────────────────────────────────────────────
# if you want a projected GeoJSON:
# centers.to_file(GEO_OUT, driver="GeoJSON")

# or to reproject back to WGS84:
centers.to_crs("EPSG:4326") \
        .to_file(GEO_OUT, driver="GeoJSON")

print("✅ Saved perfectly‐tessellated, rotated patches to:", GEO_OUT)

Rotatiehoek t.o.v. x-as: -23.67°
Bounding boxes with GPS coordinates saved to 'C:\Users\Sander\OneDrive - UGent\Semester_2\Masterproef\Thesis_ML\Roboflow\Raw_July\All_results\ViT\metadata_full_July_updated_coord.csv'
✅ Saved perfectly‐tessellated, rotated patches to: C:\Users\Sander\OneDrive - UGent\Semester_2\Masterproef\Thesis_ML\Roboflow\Raw_July\All_results\ViT\metadata_full_July_updated_coord.geojson
